# Agents

In [ ]:
pip install ollama

In [ ]:
#!ollama serve

## Test simple

In [2]:
from litellm import completion

response = completion(
    model="ollama/gemma3", 
    messages=[{ "content": "respond in 20 words. who are you?","role": "user"}], 
    api_base="http://localhost:11434"
)
print(response.choices[0].message.content)


I’m Gemma, a large language model created by the Gemma team at Google DeepMind. I’m an open-weights model!


## Test agents

In [3]:
from crewai import Agent, Task, Crew
from langchain_community.llms import Ollama

# Inicializamos el modelo local
ollama_model = Ollama(model="ollama/gemma3", base_url="http://localhost:11434")

# Creamos un agente con CrewAI
agent = Agent(
    role='Analyst',
    goal='Analizar el texto y detectar sesgos de género',
    backstory="Eres un experto en igualdad y lenguaje inclusivo.",
    llm=ollama_model
)

# Creamos una tarea
task = Task(
    description="Analiza el siguiente texto: 'Las mujeres son malas conductoras.'",
    expected_output="Un análisis del sesgo de género en el texto.",
    agent=agent
)

# Ejecutamos el crew (puede ser un solo agente o varios)
crew = Crew(agents=[agent], tasks=[task])
result = crew.kickoff()
print(result)

/tmp/ipykernel_822382/549249215.py:5: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  ollama_model = Ollama(model="ollama/gemma3", base_url="http://localhost:11434")


El texto "Las mujeres son malas conductoras" exhibe un sesgo de género muy marcado y perjudicial. La afirmación generaliza y estereotipa a las mujeres, atribuyéndoles una cualidad negativa (ser malas conductoras) basándose únicamente en su género. Este tipo de afirmación perpetúa y refuerza prejuicios sexistas, ya que no se basa en hechos ni en la diversidad de habilidades y experiencias individuales de las mujeres.  La generalización es injusta, discriminatoria y contribuye a la construcción de estereotipos negativos que limitan las oportunidades y el reconocimiento de las mujeres en el ámbito de la conducción. Es un ejemplo claro de lenguaje sexista que debe ser rechazado y corregido.


In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate

# === 1. Modelo local ===
ollama_model = ChatOllama(model="gemma3", base_url="http://localhost:11434")

# === 2. Variables ===
VARIABLES = [
    'cita_textual_titular', 'genero_nombre_propio_titular', 'genero_periodista',
    'genero_personas_mencionadas', 'nombre_propio_titular', 'personas_mencionadas', 'tema'
]

CITA_TITULAR = {'0': 'No', '1': 'Sí'}
GENERO_NOMBRE_PROPIO_TITULAR = {'1': 'No hay', '2': 'Sí, hombre', '3': 'Sí, mujer', '4': 'Sí, mujer y hombre'}
GENERO_PERIODISTA = {'1': 'Masculino', '2': 'Femenino', '3': 'Mixto', '4': 'Ns/Nc', '5': 'Agencia/otros medios', '6': 'Redacción', '7': 'Corporativo'}
GENERO_PERSONAS_MENCIONADAS = {'1': 'No hay', '2': 'Sí, hombre', '3': 'Sí, mujer', '4': 'Sí, mujer y hombre'}
TEMA = {'1': 'Científica/Investigación', '2': 'Comunicación', '3': 'De farándula o espectáculo', '4': 'Deportiva', '5': 'Economía', '6': 'Educación/cultura', '7': 'Empleo/Trabajo', '8': 'Empresa', '9': 'Judicial', '10': 'Medioambiente', '11': 'Policial', '12': 'Política', '13': 'Salud', '14': 'Social', '15': 'Tecnología', '16': 'Transporte', '17': 'Otros'}

# === 3. Esquema ===
schemas = [
    ResponseSchema(
        name=var,
        description="Objeto con 'codigo' (string) y 'evidencia' (lista de fragmentos textuales que justifican el valor)."
    )
    for var in VARIABLES
]

output_parser = StructuredOutputParser.from_response_schemas(schemas)
format_instructions = output_parser.get_format_instructions()

# === 4. Prompt ===
template_parts = [
    "Analiza el siguiente texto periodístico y clasifícalo según las variables de CONTENIDO_GENERAL.",
    "",
    "Texto:",
    "{texto}",
    "",
    "Para cada variable, devuelve un objeto con:",
    '- "codigo": el número correspondiente según las tablas (no inventes etiquetas ni palabras).',
    '- "evidencia": lista de fragmentos textuales del texto que justifican el valor.',
    "",
    "Usa *exclusivamente* los siguientes valores numéricos:",
    f"CITA_TITULAR = {list(CITA_TITULAR.keys())}",
    f"GENERO_NOMBRE_PROPIO_TITULAR = {list(GENERO_NOMBRE_PROPIO_TITULAR.keys())}",
    f"GENERO_PERIODISTA = {list(GENERO_PERIODISTA.keys())}",
    f"GENERO_PERSONAS_MENCIONADAS = {list(GENERO_PERSONAS_MENCIONADAS.keys())}",
    f"TEMA = {list(TEMA.keys())}",
    "",
    "Ejemplo de formato esperado:",
    "{{",
    '  "cita_textual_titular": {{"codigo": "1", "evidencia": ["\'María López presentó un estudio\'"]}},',
    '  "genero_personas_mencionadas": {{"codigo": "4", "evidencia": ["María López", "Juan Pérez"]}}',
    "}}",
    "",
    "{format_instructions}"
]

template = "\n".join(template_parts)

prompt = PromptTemplate(
    template=template,
    input_variables=["texto"],
    partial_variables={"format_instructions": format_instructions},
)

# === 5. Texto de prueba ===
text = """La científica María López presentó un nuevo estudio sobre inteligencia artificial,
pero el titular mencionó solo a su colega Juan Pérez."""

_input = prompt.format_prompt(texto=text)
output = ollama_model.invoke(_input.to_string())
structured = output_parser.parse(output.content)
print(structured)


{'cita_textual_titular': {'codigo': '1', 'evidencia': ['La científica María López presentó un nuevo estudio']}, 'genero_nombre_propio_titular': {'codigo': '1', 'evidencia': ['La científica María López presentó un nuevo estudio']}, 'genero_periodista': {'codigo': '7', 'evidencia': []}, 'genero_personas_mencionadas': {'codigo': '4', 'evidencia': ['María López', 'Juan Pérez']}, 'nombre_propio_titular': {'codigo': '1', 'evidencia': ['María López']}, 'personas_mencionadas': {'codigo': '4', 'evidencia': ['María López', 'Juan Pérez']}, 'tema': {'codigo': '16', 'evidencia': ['inteligencia artificial']}}


In [5]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

s1 = "La científica María López presentó un nuevo estudio"
s2 = "María López se viste de rojo"



sim = util.cos_sim(model.encode(s1), model.encode(s2)).item()
print(sim)


0.7063212394714355
